<a href="https://colab.research.google.com/github/p4arth/NLP/blob/main/bilstmfake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd 
df = pd.read_csv('/content/drive/MyDrive/LstmFake/train.csv')
test = pd.read_csv('/content/drive/MyDrive/LstmFake/train.csv')

In [2]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
df=df.dropna()

In [4]:
x=df.drop('label',axis=1)
y = df['label']

In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [6]:
voc_size = 5000

In [7]:
messages=x.copy()
messages.reset_index(inplace=True)

In [8]:
import nltk
nltk.download('stopwords')
import re
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0,len(messages)):
  review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = " ".join(review)
  corpus.append(review)

In [10]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [11]:
onehot = [one_hot(words,voc_size) for words in corpus]
onehot

[[1507, 554, 3806, 4362, 3872, 1628, 4092, 3414, 4939, 4257],
 [521, 2449, 3777, 352, 416, 745, 4770],
 [4936, 1759, 584, 3033],
 [3517, 929, 1414, 1224, 1236, 3848],
 [1776, 416, 3875, 1480, 4847, 1896, 416, 985, 3462, 4046],
 [4585,
  341,
  3084,
  4505,
  4494,
  2548,
  1623,
  2392,
  855,
  4885,
  4503,
  585,
  1997,
  1396,
  4770],
 [1259, 3519, 1343, 3848, 4709, 662, 2817, 407, 1342, 1820, 4487],
 [4420, 4328, 2194, 4732, 3402, 837, 2548, 3840, 1342, 1820, 4487],
 [1902, 1765, 4351, 4437, 86, 736, 3861, 1921, 2548, 2863],
 [2495, 1226, 3365, 2701, 827, 1248, 688, 1707],
 [212, 4459, 974, 1686, 4029, 1196, 1020, 2085, 3050, 3043, 843],
 [1224, 1, 3872, 736, 2548, 3402],
 [1555, 2542, 4079, 2804, 4063, 4658, 1824, 1565, 3082],
 [4403, 702, 2239, 564, 4777, 880, 2722, 1342, 1820, 4487],
 [2126, 4770, 3521, 18, 1480, 1342, 1820, 4487],
 [201, 1819, 1500, 4788, 4643, 2381, 4208, 1309, 1176, 3672],
 [4567, 2178, 2449],
 [327, 3171, 2920, 941, 2548, 297, 4080, 4770],
 [4608, 4123,

In [12]:
sent_length = 20
embedded = pad_sequences(onehot, padding='pre', maxlen=sent_length)

In [13]:
from tensorflow.keras.layers import Dropout
embeding_features = 50
model = Sequential()
model.add(Embedding(voc_size,embeding_features, input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 50)            250000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               60400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 310,501
Trainable params: 310,501
Non-trainable params: 0
_________________________________________________________________
None


In [28]:
from tensorflow.keras.layers import Bidirectional
embeding_features = 100
model1 = Sequential()
model1.add(Embedding(voc_size,embeding_features, input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 100)           500000    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               160800    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 201       
Total params: 661,001
Trainable params: 661,001
Non-trainable params: 0
_________________________________________________________________
None


In [29]:
import numpy as np
x_final = np.array(embedded)
y_final = np.array(y)

In [30]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x_final, y_final, test_size=0.33, random_state=42)

In [31]:
model1.fit(x_train,y_train,validation_data = (x_test,y_test), epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 8s 24ms/step - loss: 0.2815 - accuracy: 0.8693 - val_loss: 0.1998 - val_accuracy: 0.9150
Epoch 2/10
192/192 [==============================] - 4s 19ms/step - loss: 0.1294 - accuracy: 0.9506 - val_loss: 0.2032 - val_accuracy: 0.9168
Epoch 3/10
192/192 [==============================] - 4s 19ms/step - loss: 0.0827 - accuracy: 0.9705 - val_loss: 0.2361 - val_accuracy: 0.9135
Epoch 4/10
192/192 [==============================] - 4s 19ms/step - loss: 0.0500 - accuracy: 0.9833 - val_loss: 0.2918 - val_accuracy: 0.9054
Epoch 5/10
192/192 [==============================] - 4s 19ms/step - loss: 0.0314 - accuracy: 0.9904 - val_loss: 0.3647 - val_accuracy: 0.9110
Epoch 6/10
192/192 [==============================] - 4s 19ms/step - loss: 0.0174 - accuracy: 0.9949 - val_loss: 0.4818 - val_accuracy: 0.9059
Epoch 7/10
192/192 [==============================] - 4s 19ms/step - loss: 0.0112 - accuracy: 0.9967 - val_loss: 0.5580 - val_accuracy: 0.9044

In [22]:
print(np.argmax(y_pred,axis=1))

[0 0 0 ... 0 0 0]


In [32]:
y_pred=model1.predict(x_test)
classes = np.argmax(y_pred,axis=1)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,classes)

array([[3419,    0],
       [2616,    0]])

In [33]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,classes)

0.5665285832642917